# Numerisches Lösen der eindimensionalen zeitabhängigen Schrödingergleichung mittels Python

In diesem Beispiel wird [`pySchrodinger`](https://github.com/jakevdp/pySchrodinger) verwendet, um die zeitabhängige Lösung der eindimensionalen Schrödingergleichung bei stationärem externen Potential zu diskutieren. Im weiteren Verlauf werden dann verschiedene Potentialparameter und Ursprungswellenformen diskutiert. 

Die Schrödingergleichung ist das quantenmechanische Gegenstück zur klassischen Formulierung der Gesamtenergie eines Systems, in dem sich der (hier eindimensionale) Ort $x(t)$ eines Teilchens wie eine Welle $\Psi(x, t)$ verhält. Es sei darauf hingewiesen, dass die Schrödingergleichung ein Postulat ist, das sich nicht direkt aus anderen physikalischen Gesetzen ableiten lässt. Formal kann die Schrödingergleichung jedoch aus der (nichtrelativistischen) Hamilton-Funktion für einen Massenpunkt

$$E={\frac {{p}^{2}}{2m}}+V({x},t)$$

hergeleitet werden, indem die klassischen Größen Energie $E$, Impuls $p$ und Ort $r$ durch die entsprechenden quantenmechanischen Operatoren gemäß dem Korrespondenzprinzip ersetzt werden: 

$$\begin{matrix}E&\rightarrow &{\hat E}&=&{\mathrm {i}}\hbar {\frac {\partial }{\partial t}}\\{{p}}&\rightarrow &{ {{\hat p}}}&=&-{\mathrm {i}}\hbar \frac{\partial}{\partial x} \\{{x}}&\rightarrow &{{{\hat x}}}&=&{{x}}\end{matrix}$$

Auf diese Weise wird im Prinzip die Hamiltonfunktion in einen Hamiltonoperator $\mathcal H(x,\hat p,t) = -\frac{\hbar^2}{2m} \frac{\partial^2}{\partial x^2}+V({x},t)$ umgewandelt, der dann auf die unbekannte Wellenfunktion $\Psi(x,t)$ angewendet werden kann. Die ursprüngliche Hamilton-Funktion lautet nun in der quantenmechanischen Schreibweise:

$$ {\mathrm {i}}\hbar {\frac {\partial \Psi(x,t)}{\partial t}} = \mathcal{H}\Psi(x,t) = -\frac{\hbar^2}{2m} \frac{\partial^2 \Psi(x,t)}{\partial x^2}+V({x},t)\Psi(x,t)$$

Wenn der Hamilton-Operator zeitunabhängig ist, also $\frac{\partial \mathcal H}{\partial t} = 0$, was in der Regel der Fall ist, wenn das Potential "stationär" ist, d.h. $V \equiv V(x)$, dann kann die zeitabhängige Schrödingergleichung mit einem Produktansatz $\Psi(x,t) = \psi(x)\phi(t)$ gelöst werden. Einsetzen von $\Psi(x,t) = \psi(x)\phi(t)$ in die obige Gleichung führt somit zu

$$\mathrm i\hbar \frac{1}{\phi(t)} \frac{\partial\phi(t)}{\partial t}=\frac{1}{\psi(x)}\mathcal{H}\psi(x)$$

Dadurch werden die Variablen voneinander getrennt. Die Gleichung kann für beliebige $x$ und $t$ nur erfüllt werden, wenn beide Seiten gleich einer Konstanten sind; diese Konstante heißt (in weiser Voraussicht) $E$. Daher gilt für die linke und rechte Seite der Gleichung 

$$\mathrm i\hbar \frac{d\phi(t)}{dt}=E\phi(t); \quad E\psi(x) = \mathcal{H}\psi(x) $$

Für den zeitabhängigen Teil handelt sich hier um eine Differentialgleichung erster Ordnung, deren Lösung nach dem allgemeinen Ansatz $\phi(t) = A \exp(Bt)$ gefunden werden kann. Durch Einsetzen von $\phi(t) = A \exp(Bt)$ ergibt sich $B=\frac{E}{\mathrm i\hbar}=-\frac{\mathrm i E}{\hbar}$. Aus der Forderung dass unser Teilchen **zur Zeit** $t$ irgendwo in $x$ zu finden sein muss erhalten wir $A$: 

$$1 \equiv \int_{-\infty}^\infty \Psi^*(x,t)\Psi(x,t) \mathrm{d}x = \int_{-\infty}^\infty\psi^*(x)\phi^*(t)\psi(x)\phi(t)\mathrm{d}x = A \int_{-\infty}^\infty\psi^*(x)\psi(x)\mathrm{d}x = A$$ 

D.h., wir können die Lösung der zeitabhängigen Schrödingergleichung leicht erhalten, indem wir die Lösung für $\psi(x)$ aus der zeitunabhängigen Schrödingergleichung nehmen $\mathcal{H}\psi(x) = E\psi(x)$ und einfach mit $\phi(t)$ multiplizieren! 

Die stationäre Schrödingergleichung ist ein Eigenwertproblem, bei dem wir nach einer Funktion $\psi(x)$ suchen, die, wenn der Hamiltonoperator auf sie angewendet wird, wieder sich selbst multipliziert mit einem Skalar ergibt. Oft ist jedoch die Suche nach einer Funktion für nichttriviale Potentiale $V(x)$ analytisch nicht mehr möglich, weshalb man auf numerische Methoden zurückgreifen muss. Als Beispiel aus der Elektronenstrukturberechnung sei das Beispiel eines Mehrelektronensystems genannt, bei dem das Potential $V \equiv V(\{\mathbf{r}^N \})$ ist und $\{\mathbf{r}^N \}$ alle dreidimensionalen Koordinaten jedes der $N$ Elektronen darstellt, die sich alle gegenseitig über die Coulomb-Wechselwirkung beeinflussen.


Im Folgenden werden wir die zeitliche Entwicklung der Schrödingergleichung für einige einfache Fälle veranschaulichen. Obwohl es für die meisten der folgenden Fälle eine analytische Lösung gibt, ist es lehrreich, die numerische Lösung mit den analytischen Lösungen zu vergleichen. Zu diesem Zweck laden wir zunächst die dafür benötigten Bibliotheken. 

In [6]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from schrodinger import Schrodinger

Wir laden hier neben den Standardbibliotheken auch `pySchrodinger`, das uns hilft, die Schrödinger-Gleichung zu animieren und numerisch zu lösen. Bevor wir loslegen können, müssen wir einige Hilfsfunktionen definieren. Zunächst benötigen wir eine Funktion, die unser Wellenpaket im Ortsraum ($x$) und Impulsraum ($k$) darstellt. Setzen Sie die Ausdrücke für die Normalisierung und den Exponentialterm so ein, dass sich ein Gaußsches Wellenpaket der Breite $a$, zentriert bei $x_0$, mit dem Impuls $k_0$ ergibt.

In [7]:
######################################################################
# Hilfsfunktionen für Gaußsche Wellenpakete
def gauss_x(x, a, x0, k0):
    """
    ein gaußförmiges Wellenpaket der Breite a, zentriert bei x0, mit dem Impuls k0
    """
    # norm = ...     # modify
    # exponent = ... # modify
    norm = (a * np.sqrt(np.pi)) ** (-0.5)                      
    exponent = -0.5 * ((x - x0) * 1. / a) ** 2 + 1j * x * k0   
    return norm * np.exp(exponent)


def gauss_k(k, a, x0, k0):
    """
    analytische Fourier-Transformation von gauss_x(x), oben
    """
    # norm = ...     # modify
    # exponent = ... # modify
    norm = (a / np.sqrt(np.pi)) ** 0.5                         
    exponent = -0.5 * (a * (k - k0)) ** 2 - 1j * (k - k0) * x0
    return norm * np.exp(exponent)

Wir brauchen auch eine Funktion, die eine rechteckige Barriere für uns erstellt:

In [11]:
######################################################################
# Hilfsfunktionen für Barriere
def theta(x):
    """
    theta function :
      returns 0 if x<=0, and 1 if x>0
    """
    x = np.asarray(x)
    y = np.zeros(x.shape)
    y[x > 0] = 1.0
    return y

def square_barrier(x, width, height):
    return height * (theta(x) - theta(x - width))

Als Nächstes müssen wir ein paar Parameter für unsere "Simulation" festlegen.

In [5]:
# specify time steps and duration
dt = 0.01
N_steps = 50
t_max = 120
frames = int(t_max / float(N_steps * dt))

# specify constants
hbar = 1.0   # planck's constant
m = 2.0      # particle mass

# specify range in x coordinate
N = 2 ** 11
dx = 0.1
x = dx * (np.arange(N) - 0.5 * N)

# specify potential
V0 = 1.5
L = hbar / np.sqrt(2 * m * V0)
a = 3 * L
x0 = -60 * L
V_x = square_barrier(x, a, V0)
V_x[x < -98] = 1E6
V_x[x > 98] = 1E6

# specify initial momentum and quantities derived from it
p0 = np.sqrt(2 * m * 0.2 * V0)
dp2 = p0 * p0 * 1. / 80
d = hbar / np.sqrt(2 * dp2)

k0 = p0 / hbar
v0 = p0 / m
psi_x0 = gauss_x(x, d, x0, k0)

NameError: name 'square_barrier' is not defined

## Aufgabe 1: Welche Einheiten wurden hier verwendet?  

... (Lösung: atomare Einheiten) 

## Aufgabe 2: Was bedeutet eine Masse von 2 in diesem Fall? 

... (Lösung: Partikel ist doppelt so schwer wie ein Elektron) 

## Aufgabe 3: Wie wirken sich verschiedene Barrieren auf die Ausbreitung aus?

### (a): Wie wurden unendlich hohe Barriere mit diesem Tool realisiert? Diskutieren sie den Unterschied zu einem endlichen Potential.

### (b): Wie lautet die Schrödingergleichung und deren Lösung für ein freies (d.h. $V(x,t)\equiv0$) Teilchen? Lösen sie die Schrödingergleichung für ein freies Teilchen numerisch. Diskutieren sie periodische Randbedingungen.

## Aufgabe 2: Wie wirken sich andere Wellenformen auf die Ausbreitung aus? 

### (a): Eine Rechteckform?

### (b): Kollision von zwei Wellenpaketen?